<a href="https://colab.research.google.com/github/esthy13/cil-intrusion-detection/blob/main/der_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DER++ for Intrusion Detection (CIC-IDS)

Minimal **working** implementation of **Dark Experience Replay++**
for class-incremental intrusion detection.

In [1]:
!git clone https://github.com/esthy13/cil-intrusion-detection
%cd cil-intrusion-detection
!git pull
# resetting the path to content to avoid issues in the rest of the notebook
%cd ..

Cloning into 'cil-intrusion-detection'...
remote: Enumerating objects: 214, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 214 (delta 26), reused 21 (delta 11), pack-reused 166 (from 1)
Receiving objects: 100% (214/214), 62.26 MiB | 23.84 MiB/s, done.
Resolving deltas: 100% (109/109), done.
/content/cil-intrusion-detection
Already up to date.
/content


In [2]:
import os
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataset

In [3]:
class IDSBaseDataset(Dataset):
    def __init__(self, root_dir, split="train"):
        """
        root_dir: path to 2017/
        split: 'train' or 'test'
        """
        csv_dir = os.path.join(root_dir, split)
        csvs = glob.glob(os.path.join(csv_dir, "*.csv"))
        assert len(csvs) > 0, f"No CSV files found in {csv_dir}"

        df = pd.concat([pd.read_csv(c) for c in csvs], ignore_index=True)

        self.classes = sorted(df["Label"].unique())
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}

        self.x = df.drop(columns=["Label"]).values.astype(np.float32)
        self.y = np.array(
            [self.class_to_idx[label] for label in df["Label"]],
            dtype=np.int64
        )

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]), torch.tensor(self.y[idx])

## Task builder

In [4]:
class RemappedSubset(Dataset):
    """
    Subset that remaps global class indices to [0..C-1]
    """
    def __init__(self, dataset, indices, class_ids):
        self.dataset = dataset
        self.indices = indices
        self.class_map = {cid: i for i, cid in enumerate(class_ids)}

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        x, y = self.dataset[self.indices[idx]]
        return x, torch.tensor(self.class_map[y.item()])


In [ ]:
def build_scenario(all_classes, splits):
    """
    all_classes: list of class names
    splits: list of ints, e.g. [1,1,1,1] or [5,5] or [2,3,5]

    Returns: list of task class lists
    """
    tasks = []
    idx = 0
    for s in splits:
        tasks.append(all_classes[idx:idx+s])
        idx += s
    assert idx == len(all_classes), "Splits do not sum to total classes"
    return tasks

## Model

In [6]:
class CILModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fe = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU()
        )
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        z = self.fe(x)
        return self.classifier(z)

    def expand_classes(self, n_new):
        old = self.classifier
        new = nn.Linear(old.in_features, old.out_features + n_new).to(device)
        new.weight.data[:old.out_features] = old.weight.data
        new.bias.data[:old.out_features] = old.bias.data
        self.classifier = new

## Replay Buffer (DER++)

In [7]:
class ReservoirBuffer:
    def __init__(self, size):
        self.size = size
        self.n_seen = 0
        self.x, self.y, self.logits = [], [], []

    def add(self, x, y, logits):
        for xi, yi, li in zip(x, y, logits):
            li = li.detach().cpu()
            if len(self.x) < self.size:
                self.x.append(xi.cpu())
                self.y.append(yi.cpu())
                self.logits.append(li)
            else:
                j = np.random.randint(0, self.n_seen + 1)
                if j < self.size:
                    self.x[j] = xi.cpu()
                    self.y[j] = yi.cpu()
                    self.logits[j] = li
            self.n_seen += 1

    def sample(self, batch_size, current_dim):
        if len(self.x) == 0:
            return None

        idx = np.random.choice(len(self.x), min(batch_size, len(self.x)), replace=False)

        bx = torch.stack([self.x[i] for i in idx]).to(device)
        by = torch.stack([self.y[i] for i in idx]).to(device)

        # PAD LOGITS
        padded_logits = []
        for i in idx:
            old_logit = self.logits[i]
            if old_logit.numel() < current_dim:
                pad = torch.zeros(current_dim - old_logit.numel())
                old_logit = torch.cat([old_logit, pad])
            padded_logits.append(old_logit)

        blogits = torch.stack(padded_logits).to(device)

        return bx, by, blogits

## DER++ Training Loop

In [8]:
def train_task(model, loader, buffer, optimizer, alpha=0.5, beta=0.5, epochs=1):
    ce = nn.CrossEntropyLoss()

    model.train()
    for _ in range(epochs):
        for x, y in loader:
            x, y = x.to(device), y.to(device)

            out = model(x)
            loss = ce(out, y)

            buf = buffer.sample(len(x), model.classifier.out_features)
            if buf:
                bx, by, blog = buf
                loss += alpha * F.mse_loss(model(bx), blog)
                loss += beta * ce(model(bx), by)



            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            buffer.add(x, y, out.detach())

In [ ]:
def evaluate(model, dataset, classes, scenario):
    model.eval()
    loader = DataLoader(build_scenario(classes, scenario), batch_size=256)
    print(loader.dataset.class_map)
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            preds = model(x).argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return correct / total if total > 0 else 0

## Run Experiment

In [9]:
!unzip cil-intrusion-detection/data/processed/2017.zip

Archive:  cil-intrusion-detection/data/processed/2017.zip
   creating: 2017/
   creating: 2017/train/
  inflating: 2017/train/portscan.csv  
  inflating: 2017/train/ssh-patator.csv  
  inflating: 2017/train/ftp-patator.csv  
  inflating: 2017/train/bot.csv      
  inflating: 2017/train/benign.csv   
  inflating: 2017/train/web-attack.csv  
  inflating: 2017/train/dos.csv      
  inflating: 2017/train/ddos.csv     
   creating: 2017/test/
  inflating: 2017/test/portscan.csv  
  inflating: 2017/test/ssh-patator.csv  
  inflating: 2017/test/ftp-patator.csv  
  inflating: 2017/test/bot.csv       
  inflating: 2017/test/benign.csv    
  inflating: 2017/test/web-attack.csv  
  inflating: 2017/test/dos.csv       
  inflating: 2017/test/ddos.csv      


In [10]:
from torch.utils.data import DataLoader

# Paths
DATA_ROOT = "2017"  # <-- folder created by unzip

# Datasets
train_dataset = IDSBaseDataset(DATA_ROOT, split="train")
test_dataset  = IDSBaseDataset(DATA_ROOT, split="test")

print(train_dataset.class_to_idx)


{'benign': 0, 'bot': 1, 'ddos': 2, 'dos': 3, 'ftp-patator': 4, 'portscan': 5, 'ssh-patator': 6, 'web-attack': 7}


In [ ]:

input_dim = train_dataset.x.shape[1]

# Task definition (example)
all_classes = [
    "benign",
    "dos",
    "ddos",
    "portscan",
    "ssh-patator",
    "ftp-patator",
    "web-attack",
    "bot"
]

# Scenario A: 1+1+1+1+1+1+1+1
tasks_1 = build_scenario(all_classes, [1]*8)

# Scenario B: 5+3
tasks_2 = build_scenario(all_classes, [5, 3])

# Scenario C: 2+3+3
tasks_3 = build_scenario(all_classes, [2, 3, 3])

# Model + buffer
model = CILModel(input_dim, len(tasks[0])).to(device)
buffer = ReservoirBuffer(size=2000)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

seen_classes = []



# training loop that works only for scenario 1+1+1+1+1...
# TODO create a build scenario method that works for any class split
for task_id, task_classes in enumerate(tasks):
    print(f"\n=== Task {task_id}: {task_classes}")

    if task_id > 0:
        model.expand_classes(len(task_classes))
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    seen_classes += task_classes

    train_loader = DataLoader(
        build_task(train_dataset, seen_classes),
        batch_size=128,
        shuffle=True
    )

    train_task(
        model,
        train_loader,
        buffer,
        optimizer,
        alpha=0.5,
        beta=0.5,
        epochs=1
    )

    acc = evaluate(model, test_dataset, seen_classes)
    print(f"Test accuracy (seen classes): {acc:.4f}")



=== Task 0: ['benign']


AttributeError: 'DataLoader' object has no attribute 'class_to_idx'